In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

In [3]:
service = Service(executable_path=r'C:/Users/Puser/Workspace/chromedriver.exe')

In [4]:
driver = webdriver.Chrome(service=service, options=chrome_options)

In [5]:
locations = ["여의도", "국회의사당", "여의나루", "더현대", "여의도한강공원", 
             "종로", "익선", "혜화", "부암동", "평창동", "삼청동", "인사동", "대학로", "경복궁", 
             "서촌", "북촌", "종각", "동묘", "광화문", "가회", "안국", "광장시장", "낙산공원", 
             "서순라길", "정동", "효자동", "덕수궁", "창신동", "창덕궁", "낙원상가", 
             "북악스카이웨이", "북악산", "사직동"]

In [6]:
hashtags = ["#승연아_우즈야_여기야 OR #신동_넌이미먹었겠지 OR #박지성_단1g도안줌 OR #제노의_맛그당어 OR #런쥔이_밥무거쒀 OR #확신해찬맛집 OR #몬베베가_몬베베에게_추천하는_맛집 OR #백현이를_위한_맛집투어 OR #황제님을위한메뉴판 OR #강다니엘_염염긋 OR #완전_장원영_스타일_아니냐구 OR #ㄷㅂㅇㅈ OR #런쥔이의_맛집내비게이션 OR #승철이_이거먹고_볼빵빵했꾸마 OR #정한아_마니머거여ㅎㅎ OR #조슈아_먹어보슈아 OR #순영아_이거_맛있어 OR #원우야_여기_테이스티 OR #민규라고치고_디너쇼해도돼 OR #도겸이도_도아할_맛집 OR #믿고먹어boo세요", 
           "#남도현_한입도안줄래 OR #재민아_밥먹어 OR #은하야_이거_맛있어 OR #지우야_맛집은여기 OR #우석아_맛집으로_우따따따 OR #도겸아_여기야_피자대동여지도 OR #잘_먹고_잘_사는_순딩이 OR #주헌아_잡솨_바바요 OR #펜타곤함께먹자 OR #백현이를_위한_레시피_공유 OR #백현이를_위한_맛집_투어 OR #민석이를_위한_카페투어 OR #민석이를_위한_빵집투어 OR #강동호_먹어봐 OR #세훈이의_여행을_떠나요 OR #백현이를_위한_일본투어 OR #먹어봤도영 OR #황제를위한메뉴판 OR #동현이를_위한_대동맛지도 OR #정우의_원데이식스밀 OR #정우의_원데이씩스밀 OR #먹다가_해찬이_생각이나신거예요 OR #마크한테만_공개 OR #성찬이의_진수성찬 OR #기현_너는맘마먹고다시시작하겠지 OR #아이유애냐미냐미 OR #더보이즈_배고플땐언제든찾아와요 OR #크리스_배고파용", 
           "#리노는_배고파 OR #창빈이는_안보겠지ㅠ OR #현이도_먹어봐 OR #한이의한입거리 OR #이_식당은_진심으로_하늘이에요 OR #여기맛있승민 OR #아기빵_크림채워 OR #eaj_말구_eatj OR #나는_너의_밥선생 OR #영현아_여기서도_7인분_어때 OR #나나나나_원필이가_먹을래 OR #도운이_밥묵엇슴까 OR #우리배는_식도로만_가 OR #이게바로_에이티즈의_맛인기라 OR #아니예요_준휘는_먹을거예요 OR #우지를삼킬순없잖아 OR #밍하오야_여기_헌하오츠 OR #THE_ATE OR #버논아_혹시여긴_맛집 OR #라즈지 OR #디노야_여기_케이밥의_미래 OR #캐럿들_여기_캐맛있어 OR #계산은_쿱스가 OR #위EAT_인EAT OR #승윤이의_맛집_PAGE OR #쭈대장과_함께", 
           "#이맛집_찐받네 OR #마이노_마이무 OR #훈슐랭_여기도가봤니 OR #야빠들도_맛있는거먹어 OR #황대인_넌_이미_단골이겠지 OR #먹으러갸자 OR #갸영철_여긴_꼭_먹어라 OR #경수야_여기가_맛집이디오 OR #곤_만나서_먹자 OR #병곤아_이거_곤나맛있어 OR #승훙이도_훙입만 #먹어보세용_희희 OR #아강이_냐미냐미 OR #현석아_이거_머거바니 OR #서기의_메뉴판 OR #삼킬듯한 OR #배지니욘나_맛집 OR #너무맛있어서픽스러짐 OR #맛있겠짛_주연앟 OR #이주연_난늘배고파_베베 OR #이재현_치킨이_다가아니야 OR #어이어이_이재현_여기다 OR #영훈아_밥좀많이먹어라 OR #재현아_차츰공복이드리울테니 OR #영훈아_빵빵하게_먹어 OR #NewFoodshot OR #최찬희_요정은_밥심이다 OR #지창민_도토리대신 OR #지창민_떡볶이말고_이건_어때 OR #뱀맛집 OR #배무맛집 OR #퉤이스티로드 OR #왕잭슨_여기_왕맛있다 OR #냠긔탱긔 OR #세훈아_이거봐라_맛있겠지 OR #드셔보세훈 OR #세훈이_세입먹을때_세프는_세그릇째 OR #세훈아_순댓국_말고_이거 OR #소원아여기가거기야 OR #맛린집 OR #은하에게_강추 OR #유주라잌썸띵투잍 OR #맛이신비해 OR #무지야여기무지맛있어 OR #여기먹어볼새롬 OR #문익아_여기서_파는거야 OR #먹잘알_종형아_이거_먹잘 OR #기석이_밥_먹을_시간 OR #재찬아_밥먹자 OR #먹어보시갱 OR #꼬르륵소리 OR #배부르다현 OR #상아야_여기_맛집_맞지", 
           "#원상아_여기_알라빙빙 OR #셔누_넌이미먹었겠지만또먹어야겠지 OR #민혁_너는이미북마크해놨겠지 OR #기현_너는맘마먹고다시시작하겠지 OR #채형원_너는세입도못먹겠지 OR #주헌_너는한입가득못먹겠지 OR #임창균_너는한끼뚝닭먹겠지 OR #웅이가_맛보면_좋겠웅 OR #우진아_맛집_추천_별거아냐 OR #동현이를_위한_대동맛지도 OR #대휘가_먹으면_아맛나 OR #지젤아밥무라 OR #윈터의_맛집을_입력해주세요 OR #닝닝이랑같이먹고싶은거 OR #지구에서여기가젤루맛있어 OR #이맛집은신이에요 OR #우리배는_식도로만_가 OR #에이티즈_우리배는_식도로만_가 OR #이게바로_에이티즈의_맛인기라 OR #이타다끼_마쓰 OR #박성화_세입컷챌린지 OR #왕맛있슨 OR #대열아_여기가맛있대열 OR #와이_렇게맛있을수가 OR #장준아나중에밥이나한끼하자 OR #태그야_맛집해시태그야 OR #승민아_여기_z지존킬z OR #봉재현_이건_먹겠지 OR #지범_맛집다녀오겠습니다 OR #주찬아_쫓겨나기전까지만먹자 OR #주찬_넌이미여기서쫓겨났겠지 OR #동동맛집센터 OR #보민아_여기가극락인가봄 OR #Y가_최고로_모시는_맛집 OR #캐럿들_여기_캐맛있어 OR #세운아_먹어보세운 OR #이맛이꾸민지생시인지 OR #맛집으로달려라하니 OR #단다니챙겨먹자 OR #난맛집을찾는_베이비 OR #혜인아_다음에꼭같이가자 OR #손영재_콧구멍_커지겠지 OR #주현아맛있겠찌 OR #슬슐랭가이드 OR #오늘은웬디여기 OR #수영아아줌마들잘먹지 OR #누가_날_맛집으로_땡긴다악 OR #연히의_냠냐미 OR #여기수윤이자린데오", 
           "#NJ맛집_맞지 OR #드댕이도_드냠드냠 OR #협아_이_디저트꼭먹어봐 OR #윤성아_혼자_먹기_아까워 OR #주창욱_넌_안먹어봤겠지 OR #동윤아_맛집은_안중요한가 OR #민서가_쏠테니_많이먹어 OR #집중집중_준호_맘마타임 OR #준호야_맛있는_거_먹으러_가차 OR #이_거맛있어_협 OR #렉스에게_가장_잘맞는_맛집은 OR #영현아_여기서도_7인분_어때 OR #경윤아_여기_찐맛이야 OR #AI_충전기 #민규_이거_먹어 OR #박성화_너는세입이면먹겠지 OR #박성화_이것좀_먹어보라고_성화 OR #캡틴홍중의_다음_먹적지 OR #김홍_중독될정도로맛있는집 OR #정윤_호불어서먹어 OR #정윤_호텔5성안부럽다 OR #정윤_호텔5성뷔페안부럽다 OR #윤둥아_이거_둥나맛있어 OR #맛이여_상상이상 OR #강여_상당히맛있는집 OR #강여상_치킨말고_이것도_먹어봐 OR #산맛집 OR #산이미식회 OR #여기도_먹어보산 OR #오늘도_잘먹었산 OR #최산을_다해_먹어봐요 OR #송민_기가막힌맛집 OR #맛있게_먹우영 OR #정우영_한입에_먹우영 OR #정우_영원히먹을수있는집 OR #쫑슐랭가이드 OR #찍먹보단쫑먹 OR #최종_호불호없는맛집 OR #최종호_먹어봐_최종보스_맛집 OR #VJ_윤호특공대 OR #김선우_맛있는걸내가놔둘리가 OR #몽아_몽먹었니 OR #캐럿들_여기_캐맛있어 OR #수아_여기_와봤수아 OR #여기와봤_수아 OR #뚜현아_진짜_여기_맛집이야 OR #뚜진맛 OR #수현아_밥먹자 OR #먹잘알람 OR #라미_완소한끼 OR #츠키_많이먹어 OR #츠키야당근말고이거 OR #수연이랑_냥냥멍냠 OR #션이랑_냠냠멍냠 OR #이거먹자_시윤아 OR #이거먹어볼래_시윤아 OR #오늘도_냠냠하루나 OR #하루나_여기맛집이야 OR #Oneday_나의까까 OR #블랙핑크_맛집사거리 OR #제니야_여기젠나마싯다 OR #채영아_눈물나는맛이야 OR #지수야_여기츄라이츄라이 OR #리사에게_가장맛난걸로줘 OR #천러야_이거_먹기좋아 OR #은광아_이거_누룽지백숙만큼_맛있어 OR #여기_도토리 OR #창섭아_맛집어디있는지알게됐됬됐됬됐다", 
           "#현식적인맛집추천 OR #이거맛있뎨 OR #창섭아_여기맛있섭 OR #남준아여기째끼럽 OR #문득생각해석진이가이걸먹어봤을까 OR #머거스트디 OR #야호바너이거먹어봤냐 OR #지민이와_밥밥밥을먹어요 OR #뷔슐랭 OR #정국아_같이먹짱 OR #효앞꾸먹 OR #병근한입 OR #휘랑둥_여긴가야해 OR #석원이도냠냠 OR #이타_다끼마스 OR #편식쟁이_원이_먹어봐 OR #현빈아_난_먹어봤지롱 OR #도환아_여기와서_먹어봐 OR #맛집을_태그 OR #크래비티_무뿌식뿌 OR #나_한입_세림이_세입 OR #앨런아_맛집으로_지금_당장_런 OR #정모야_맛집_뒤집자 OR #우빈아_데굴데굴_냠 OR #원진아_함번만_먹어봐 OR #미니미니미니_강쥐는먹으면안돼용 OR #형준아_내랑_밥물래 OR #태영이도_맛집_태워가영 OR #성민이_입에_안성맞춤 OR #엔진예절 OR #엔진이_커넥해주는_맛집 OR #가든_맛집 OR #정원이_볼살로_레쭈고 OR #맛있떠_누가만들었어이거 OR #정원이의_식사랜드 OR #당연함_이희승이먹어야함 OR #희승아_라면말고_이건어때 OR #오늘의_제이 OR #맛집_기대해주셔도_감사합니다 OR #재윤이에게_메뉴추천 OR #제이크_요기요 OR #성훈이_입속으로_트리플악셀 OR #선우도_가고싶어하는_맛집 OR #선우야_이것도_맛있을거예요 OR #니시무라_마니무라 OR #민석아금강산도식후경 OR #민석아_여기가_맛집이슈 OR #준면아_여기_준맛탱 OR #백현아_먹짱일기_적짱 OR #됴슐랭 OR #경수야_먹어됴 OR #종인아_잘먹을게 OR #종인아또나만먹는거지 OR #맛집_추천해주세훈", 
           "#하영아_진짤루_맛집잉엥용 OR #맛집을_규와앙 OR #지원아밥메건니 OR #지선아_누가1인분만먹고살아_미친거지 OR #역시_대나무가_아니면_곤란한가요_더여나 OR #우리_챙람쥐_식량저장소 OR #나경아_여기맛집웅앵 OR #꿀깅아_여기_꿀발랐다 OR #이식당놓쳐서후회한다면후회한시간을후회할거잖아후회하기싫음후회할일들을후회하기전에_이거먹어봐 OR #지나니_이거맛나니 OR #바비야밥이왔어요 OR #윤형아_여기맛집이햫 OR #준회야_여기밥많이주내 OR #찬우야여기맛집이_왔쨔뇨 OR #김동동_일단밥부터먹혁 OR #어_리안이가_리안이가 OR #미래에_갔다왔는데_맛집_돼_있던_맛집 OR #도도의_맛집여행 OR #카엘아_여기야 OR #동표야_넌이거못먹지 OR #시영이가_생각나면_올릴게요 OR #장유빈_멋있고_귀여운_막내온탑 OR #재혀니_맛있게머겅 OR #정우의_원데이씩스밀 OR #도영이의유맹맛집 OR #태용아_먹어보태용 OR #존맛툥구리 OR #JMTforTAEIL OR #먹태일_기릿 OR #태일아_먹어봐 OR #맛있었쟈니 OR #먹으러가쟈니 OR #먹어봤쟈니 OR #체리모토랑_맛집_같이_하겠다 OR #윈윈_먹으라고_알랑방구 OR #먹으면_서로_윈윈 OR #悠太_味확인맛집 OR #유타_味확인맛집 OR #우와_저맛집유맹해 OR #마크한테만공개 OR #나나의_맛집탐방기 OR #나나챌린지 OR #맛집사냥쿤 OR #이거맛있쿤 OR #食中食_TEN OR #모든식당텐기다려줘방문할게 OR #샤오쥔_광둥타코말고_이거먹어봐 OR #덕준아_이세상엔_맛있는게많다 OR #헨드리_맛도리 OR #헨드리_안먹으면_후회한드리 OR #양양아_념념하자 OR #양양_먹어봐_얌얌 OR #먹어봤_쇼타로 OR #튼튼하게_먹지성 OR #승짱JMT니까먹어봐 OR #차훈_넌어차피_안먹겠지 OR #재현아_맛있겠지_김재현이요 OR #회승아_만들수있니_집밥요정 OR #동동아_먹어봐 OR #오소스_소스에_찍어_먹어 OR #알리오소스_아니고_저스트_오소스 OR #레이븐이_가봐야할_이쁜_맛집 OR #서호야_호호불어먹어 OR #궈낙아_이거머궈봐 OR #건학아_이거만먹고_운동할게 OR #건희를_위한_추천맛집", 
           "#환웅아이거먹고사람될게 OR #내동주고_사먹은_맛집후기 OR #용훈이_얼굴만큼_기대하고가도되는곳 OR #강건너_현구맛집 OR #kang_981124 OR #ㅎㄹㅇㅌㄱㅂ OR #하린아_자셔봐 OR #동명이_배고파 OR #기욱이의_볼주머니_저장창고 OR #효진이는_앞으로도_꾸준히먹자 OR #효앞꾸먹 OR #효지나마이크내리고젓가락들어 OR #이션같이찌자 OR #이같찌 OR #창윤아내가다계산할게먹기만해 OR #먹어야살어 OR #이승준먹방대결 OR #어스_맛집_여기있으 OR #수박민균 OR #민균이를위한야미푸드 OR #엇대먹 OR #꺼야미야미 OR #유토와_같이먹자 OR #맛집소개해줄게Ü OR #인성아나이거오분순삭 OR #영균아_뭐라도_먹어봐 OR #저는잘먹을수있어요 OR #얘두다나이거먹음 OR #김석우_입으로_떼구르르 OR #옴뇸뇸소리안나요 OR #태양아_이거어때_난좋은데 OR #함께먹어주호 OR #영빈아왜먹질못하니 OR #기범아_나혼자먹는거_아니야 OR #종현아_우린_대왕쁘띠가될거야 OR #민호야_식단관리_이겨내 OR #태민_DO_EAT_BABY OR #찡기야_맛있겠찡 OR #릴프릭이_함냐함냐함 OR #크리스_배고파용 OR #창빈이는_안보겠지ㅠ OR #현이도_먹어봐 OR #한이의한입거리 OR #이_식당은_진심으로_하늘이에요 OR #현이도_먹어봐 OR #쏟아지는_맛집 OR #비니의맛집_마시쪄마시쪄 OR #수민아이거먹어 OR #수민아_이거먹어 OR #시은이의꿀맛보장 OR #아이사장님뭘좀아네 OR #맛집이세은 OR #맛집이윤 OR #재이야여기짱맛있어 OR #냠냠쭌쭌 OR #복숭아맛집 OR #성준아_여기_같이_오자 OR #오냠오냠 OR #휘로록짭짭 OR #낑깡현수 OR #휘풍당당_맛집_나가신다 OR #현석이_입맛이에 OR #지훈아_여기가봐라 OR #욧집 OR #준규_메롱 OR #마시호_여기가보시호", 
           "#재혁아_바나나우유말고_이것도먹어봐 OR #사히맛집태그언제생겨 OR #사히밥 OR #예담아_담아놔 OR #이야아마시따아도영아 OR #정우야_여기_먹으러_꼭와 OR #라마x3 OR #정환아_여기맛있소 OR #쩝쩝박사_말랑이에게 OR #지효야나먹어쪄 OR #사나없이먹으나마나 OR #최연준_맛집1등어디야 OR #연도먹_연준이도먹어 OR #연준아_여기_으악으엑 OR #수빈아_먹어봐_마수리수리 OR #범규야_제발_한입만_먹어봐 OR #태현아_한번_맛볼테리 OR #태현이의_소식먹방 OR #강이사님_드셔보세요 OR #휴닝이는_맛있는거_먹고심분딩 OR #휴닝이_배고파요 OR #휴닝아먹어봐_백퍼카이취향 OR #투모로우바이투게더_먹어봐 OR #허나아_밥먹자 OR #호영씨_아아아아 OR #민찬아_만찬이야 OR #계현보장맛집 OR #여노야_밥은_먹고_다니니 OR #용왕님_기미상궁_마쳤어용승 OR #이게내마지막밥이라고 OR #너에게내밥상을줄게 OR #상엽아_다람쥐는_이거_못먹겠지 OR #휘연이와_함께하고싶은_곳 OR #지냥이_배고파", 
           "#kcal_플랙스 OR #펜타곤_같이먹자 OR #홍석_넌_먹으면_근손실오겠지 OR #여기바로고민보다고_신원 OR #펜타곤에서제일베부른정우석 OR #형구야_여기_완전_기가막키노 OR #후_이거먹으러_와라_회택아 OR #개밥보다맛있는몽몽이밥 OR #여기넘맛있어유_또올래 OR #이거먹고다이어트_조지노 OR #예_난여기로정했다 OR #택운이를_위한_뮤슐랭가이드 OR #학연이가_뇸뇸_먹어볼까욘 OR #라비_맛집_도와드려요 OR #갈비찜을밥위에얹어주세횩 OR #쟌이뭐먹어 OR #밥먹었어요_아니요 OR #고뚠아_이거먹어봐 OR #도시야_이거_억수로_맛있데이 OR #수안이가_행복해지려면_이걸먹어봐 OR #이거무디맛있어 OR #사회인먹짱클럽 OR #THIS_IS_EAT OR #명준이_아가배_지킴단 OR #진우야_이거다_이거맛있지누 OR #빈아_운동및식단_포기했다 OR #돌잔치_음식추천 OR #로하_이거먹으러_왔따나 OR #은우야_인누와_같이먹차 OR #두준아나두먹을래 OR #식샤님의_깨끗한_밥그릇 OR #식샤님_블로그_올리셨나요 OR #식샤님_오늘은_뭐드셨어요 OR #맛섭아_요집이야 OR #근손실도_이기구왕 OR #동운아_밥은먹고다니냐 OR #전영중_넌이미먹었겠지 OR #상호_밥_무라 OR #찬양이도_찬양하는맛집투어 OR #인간태풍_상륙 OR #맛이상당히준수하네요 OR #박병찬의_밥반찬 OR #누가계속맛집가래 OR #최종수_오른쪽으로가다가멈춰서젓가락질_막아봐 OR #너생각보다맛집이구나 OR #밥묵자부기 OR #아_하세혁 OR #종혁아_밥먹었니 OR #아빠_이거사줘 OR #맛있으면박솸번주쉐이 OR #가슴이웅덩해지는맛 OR #하오_늘도잘먹었다 OR #한빈이_입속으로_올인해 OR #석메추 OR #태래비전출연맛집 OR #우_리키_리만_먹자 OR #규비니랑_와규와규 OR #밥건욱 OR #유진아복숭아는에피타이저구 OR #먹다가_해찬이_생각이_나신거예요 OR #성규야_여기가_바로_맛집인뎁쇼 OR #유정이가_좋아하는_맛집_좀_추천해줄래 OR #밥먹었나영 OR #나영이랑_더_빠르게_맛집_찾아볼래요", 
           "#foodforboo OR #션프는셔누사랑만먹을게_갈비찜 OR #주학년_먹는데돈아끼지말자 OR #주학년_푸드플렉스 OR #학신 OR #금동아_이거먹고_세계정복 OR #상상그이상의_맛집 OR #여보세요_상연아_나_맛집이야_끊어 OR #쩸냠냥 OR #상프들_맛집은_신중해야해 OR #그거먹고_배가찬희 OR #단서_말고_맛집_찾자 OR #특별한_음식_특이에게_추천하기 OR #희철아_고양이는_이런거_못먹겠지 OR #맛집_규게좋은거야 OR #규가수에_규팬 OR #규현을_위한_맛집_만나러_간다 OR #내_위장이_움찔했던_순간 OR #고마운_려욱이에게_맛집추천 OR #은혁아_맛집_땡겨 OR #맛집으로_이동해 OR #설원대짜하나주세요 OR #케빈_내밥의반을줄게 OR #케빈_설렐때만_맛집이니까 OR #현재야_치킨이닭 OR #아땡마랑_같이먹자 OR #콥이는_콥빼기로주세요 OR #창균아_오늘의_일반식 OR #파랑한끼 OR #우무이 OR #Black항상잘먹음 OR #밥을먹는거아니었어감동을먹고있었어 OR #태어나서밥안먹어본사람처럼밥을먹었어 OR #냠냠쭉쭉 OR #이거먹으러카즈하 OR #자윤아여기맛있어 OR #태웅이를위한잠이확깨는맛집 OR #여기가_해피이한_맛집 OR #우즈야_무즈는_이거먹어 OR #더비예절 OR #캠든아_이거_캠맛있다 OR #원_없이먹어_빈아 OR #원빈이는기타쳐_난주문벨칠게 OR #실버램지의_추천 OR #민규야원우야_여기비럴스윗 OR #아땡마_잘먹지 OR #호랑이는_못먹겠지 OR #런쥔이왔당 OR #탱구야_여기존맛탱구리 OR #나인이가_아마_먹을_수_있을걸 OR #먹둥지말둥지 OR #재민아_우리_먹을때_같이가 OR #아니에요_준휘는_먹을거에요 OR #백현아오늘은 OR #돼지성_단1g도안줌 OR #인기현상_맛집해시_뭐였더라 OR #효랑단이_맛집_알려줄게효 OR #여기가_맛집으론_최고지호 OR #모승맛인지_궁금하지 OR #큥튜브_먹방 OR #황제님을_위한_메뉴판 OR #도나쓰_냠 OR #먹태일기릿 OR #밥먹었수호"]

In [7]:
hashtags1 = []
for hashtag in hashtags:
    hashtag = hashtag.replace(" ", "%20")
    hashtag = hashtag.replace("#", "%23")
    hashtags1.append(hashtag)
print(hashtags1)

['%23승연아_우즈야_여기야%20OR%20%23신동_넌이미먹었겠지%20OR%20%23박지성_단1g도안줌%20OR%20%23제노의_맛그당어%20OR%20%23런쥔이_밥무거쒀%20OR%20%23확신해찬맛집%20OR%20%23몬베베가_몬베베에게_추천하는_맛집%20OR%20%23백현이를_위한_맛집투어%20OR%20%23황제님을위한메뉴판%20OR%20%23강다니엘_염염긋%20OR%20%23완전_장원영_스타일_아니냐구%20OR%20%23ㄷㅂㅇㅈ%20OR%20%23런쥔이의_맛집내비게이션%20OR%20%23승철이_이거먹고_볼빵빵했꾸마%20OR%20%23정한아_마니머거여ㅎㅎ%20OR%20%23조슈아_먹어보슈아%20OR%20%23순영아_이거_맛있어%20OR%20%23원우야_여기_테이스티%20OR%20%23민규라고치고_디너쇼해도돼%20OR%20%23도겸이도_도아할_맛집%20OR%20%23믿고먹어boo세요', '%23남도현_한입도안줄래%20OR%20%23재민아_밥먹어%20OR%20%23은하야_이거_맛있어%20OR%20%23지우야_맛집은여기%20OR%20%23우석아_맛집으로_우따따따%20OR%20%23도겸아_여기야_피자대동여지도%20OR%20%23잘_먹고_잘_사는_순딩이%20OR%20%23주헌아_잡솨_바바요%20OR%20%23펜타곤함께먹자%20OR%20%23백현이를_위한_레시피_공유%20OR%20%23백현이를_위한_맛집_투어%20OR%20%23민석이를_위한_카페투어%20OR%20%23민석이를_위한_빵집투어%20OR%20%23강동호_먹어봐%20OR%20%23세훈이의_여행을_떠나요%20OR%20%23백현이를_위한_일본투어%20OR%20%23먹어봤도영%20OR%20%23황제를위한메뉴판%20OR%20%23동현이를_위한_대동맛지도%20OR%20%23정우의_원데이식스밀%20OR%20%23정우의_원데이씩스밀%20OR%20%23먹다가_해찬이_생각이나신거예요%20OR%20%23마크한테만_공개%20OR%20%23성찬이의_진수성찬%20OR%20%23기현_너는맘마먹고다시시작하겠지%20OR%20%23아이

In [14]:
url = "https://twitter.com/search"
driver.get(url)
time.sleep(5)

In [ ]:
for location in locations:
    tweets = []
    
    for hashtag in hashtags1:
        url = "https://twitter.com/search?q={0}%20{1}&src=typed_query".format(location, hashtag)
        driver.get(url)
        time.sleep(5)
        
        before_location = driver.execute_script("return window.pageYOffset")

        while True:

            driver.execute_script("window.scrollTo(0,{})".format(before_location + 2000))
            time.sleep(3)

            after_location = driver.execute_script("return window.pageYOffset")
            
            if before_location == after_location:
                break
                    
            else:
                html_content = driver.page_source
                soup = BeautifulSoup(html_content, 'html.parser')
                posts = soup.find_all(attrs={"data-testid": "tweetText"})
            
                for post in posts:
                    if post.text not in tweets:
                        tweets.append(post.text)
                        print(post.text)
                        print("-" * 50)
                    else:
                        pass
                        
                before_location = driver.execute_script("return window.pageYOffset")
            
    df = pd.DataFrame(tweets, columns=['tweet'])
    df.to_csv("twitter_{0}.csv".format(location), index=False)

 여의도 오복수산

사케동 조오오온맛 
진심 배 터질 거 같아 •• 
#도겸이도_도아할_맛집
--------------------------------------------------
#원우야_여기_테이스티
브라운돈까스 여의도점 ◠‿◠ kbs 올 일 있는 돌덕 칭구들 가볍게 먹기 조아용 사실 목적은

역시 돈까스는 부드러운 안심이지
ㄴ이건 두툼한 등심입니다만,
--------------------------------------------------
#강다니엘_염염긋
압구정(고복수산) 여의도, 연남동 오복수산 일케 많이 갔어 나 쫌 찐이지 않니 존나 맛있다 얘드라 게딱지도 꼭 먹고 장어덮밥(원래안좋아하는데)도 맛있엇음
--------------------------------------------------
#ㄷㅂㅇㅈ #지창민_도토리대신
#김선우_맛있는걸내가놔둘리가
#NewFoodShot
여의도 사위식당 

#논산큐프 합체쇼.,~ ♡
--------------------------------------------------
#승연아_우즈야_여기야
여의도 카레오 

점심시간에 웨이팅 쩔어서 (우즈??라는 분의 WAITING이 더 쩔긴함) 힘들긴하지만 맛있어요.....
--------------------------------------------------
#승연아_우즈야_여기야
여의도 셋째집 

여태 먹어본 냉삼 중 제일 맛있엇습니달라....
꽃삼겹이랑 치즈삼겹 같이 시켜서 번갈아드세요~~~~ 기본찬도 짱맛있음
--------------------------------------------------
여의도 낭만솥뚜껑

고기빨리먹고싶어서 우삼겹시키는여자들
김치가진짜개맛있음
비빔국수 강추

#원우야_여기_테이스티
#우지를삼킬순없잖아
#조슈아_먹어보슈아
--------------------------------------------------
육회관포차- 샛강 여의도 

진짜 다 맛있어서 먹는내내 침 흘리기...
강추강추해욥 ,,

#몬베베가_몬베베에게_추천하는_